In [19]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import os

# 模型加载
model_name = "/kaggle/input/llama3-8b-instruct/transformers/default/1/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    output_hidden_states=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

### 方法一

In [38]:
XP = "If you enter a building from the east side, walking west, and then take two rights and a left down corridors, which direction are you facing?"

def get_hidden_states(prompt, layer_idx=-1):
    """获取指定层的hidden states（序列最后一个token）"""
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    
    # 获取指定层的hidden states（序列最后一个token）
    hidden_states = outputs.hidden_states[layer_idx]
    return hidden_states[0, -1, :].cpu().numpy()  # 形状: [hidden_size]

def save_hidden_states(hs_dict, filename="hidden_states_data.pkl"):
    """保存hidden states数据"""
    with open(filename, 'wb') as f:
        pickle.dump(hs_dict, f)
    print(f"✓ Hidden states已保存至 {filename}")

# --------------------------
# 生成初始显式跳步路径（仅1个prompt）
# --------------------------
print("\n生成初始显式跳步路径（仅1个prompt）...")

# 仅用1个优化后的prompt，无需多个选择
single_prompt = f"""### Task: Generate Direction Reasoning Result
### Precondition (Xp): {XP}
Requirements:
- Output format: "Initial Condition → [direction]"
- [direction] must be: east, west, north, or south
- No explanations, just the formatted answer

Answer: Initial Condition → """

print(f"使用的Prompt: {single_prompt}")

# 生成模型响应
inputs = tokenizer(single_prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=80,
        num_return_sequences=1,
        do_sample=False,  # 确定性生成
        temperature=0.0,
        repetition_penalty=1.5,
        output_hidden_states=True,
        return_dict_in_generate=True
    )

# 提取核心响应（仅### Your Output:后的内容）
full_response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
core_response = full_response.split("### Your Output:")[-1].strip() if "### Your Output:" in full_response else full_response.strip()
print(f"模型核心响应: {core_response}")

# 提取hidden states（最后一层最后一个token，直接保存该结果）
last_step_hidden = outputs.hidden_states[-1][-1]  # [1, seq_len, hidden_dim]
hs_A = last_step_hidden[0, -1, :].cpu().numpy()

# 保存数据（仅1个生成结果，无需选择）
initial_data = {
    'hs_A': hs_A,
    'used_prompt': single_prompt,
    'model_response': core_response
}

save_hidden_states(initial_data, "initial_hidden_states_direction_single.pkl")  

print(f"\n✓ 初始显式跳步路径生成完成")
print(f"hs_A shape: {hs_A.shape}")
print(f"✓ 结果已直接保存")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



生成初始显式跳步路径（仅1个prompt）...
使用的Prompt: ### Task: Generate Direction Reasoning Result
### Precondition (Xp): If you enter a building from the east side, walking west, and then take two rights and a left down corridors, which direction are you facing?
Requirements:
- Output format: "Initial Condition → [direction]"
- [direction] must be: east, west, north, or south
- No explanations, just the formatted answer

Answer: Initial Condition → 
模型核心响应: ### Task: Generate Direction Reasoning Result
### Precondition (Xp): If you enter a building from the east side, walking west, and then take two rights and a left down corridors, which direction are you facing?
Requirements:
- Output format: "Initial Condition → [direction]"
- [direction] must be: east, west, north, or south
- No explanations, just the formatted answer

Answer: Initial Condition →  South


Explanation:

1. Enter Building - East Side -> You're initially heading towards West.
2.Walking to WEST 
3.Take Two Rights Down Corridors->You'

In [36]:
XP = "If you enter a building from the east side, walking west, and then take two rights and a left down corridors, which direction are you facing?"

# 定义预设的推理路径（包含完整推理和不同程度的跳步）
reasoning_paths = {
    "path_1_full": [
        "Let's think step by step: ",
        "You start by entering from the east, walking west. So initial direction: west. ",
        "First right turn: west → north. ",
        "Second right turn: north → east. ",
        "Then a left turn: east → north. ",
        "So the final direction is north."
    ],
    
    "path_2_full": [
        "Step by step reasoning: ",
        "Starting direction: west (since entering east and walking west). ",
        "First right: west turns to north. ",
        "Second right: north turns to east. ",
        "Left turn from east: facing north. ",
        "Final direction: north."
    ],
    
    "shortcut_partial": [
        "Initial direction: west. ",
        "Two rights and a left: west → north → east → north. ",
        "Final direction: north."
    ],
    
    "shortcut_direct": [
        "The final direction is north."
    ]
}

def generate_path_hidden_states():
    """为每个预设路径生成hidden states（多次生成以获得稳定性）"""
    path_hs_dict = {}
    path_responses = {}
    
    for path_name, path_steps in reasoning_paths.items():
        print(f"\n生成路径: {path_name}")
        
        # 构建完整提示词
        if isinstance(path_steps, list):
            path_text = "".join(path_steps)
        else:
            path_text = path_steps
            
        full_prompt = f"{XP}\n{path_text}"
        
        # 多次生成以获得稳定性
        hs_list = []
        response_list = []
        
        for gen_idx in range(3):  # 每个路径生成3次
            inputs = tokenizer(full_prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
            
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=20,  # 对于预设路径，只需要少量额外生成
                    num_return_sequences=1,
                    do_sample=True,
                    temperature=0.7,
                    output_hidden_states=True,
                    return_dict_in_generate=True
                )
            
            # 获取响应和hidden states
            response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
            response_list.append(response)
            
            # 提取hidden states
            all_hidden_states = outputs.hidden_states
            last_layer_hs = all_hidden_states[-1][-1]
            hs_vector = last_layer_hs[0, -1, :].cpu().numpy()
            hs_list.append(hs_vector)
            
            print(f"  生成 {gen_idx+1}: {response[-50:]}...")  # 显示最后50个字符
        
        path_hs_dict[path_name] = hs_list
        path_responses[path_name] = response_list
    
    return path_hs_dict, path_responses

# 生成所有路径的hidden states
print("开始生成预设推理路径的hidden states...")
path_hs_dict, path_responses = generate_path_hidden_states()

# 构建完整结果并保存
complete_results = {
    'path_hidden_states': path_hs_dict,
    'path_responses': path_responses,
    'reasoning_paths_definitions': reasoning_paths,
    'XP': XP  # 额外保存问题，方便后续查看
}

# 保存结果
save_hidden_states(complete_results, "method1_direction_complete_results.pkl")
print(f"\n✅ Cell 1完成！")
print(f"✅ 所有生成的hidden states已保存至 method1_direction_complete_results.pkl")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


开始生成预设推理路径的hidden states...

生成路径: path_1_full


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


  生成 1: r: north. I hope it is correct. Please let me know...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

  生成 3: s see if we can find a counterexample. Suppose you...

生成路径: shortcut_direct


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


  生成 1: uilding from the east side, so you are facing east...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


  生成 2:  when you enter the building.
You take two rights,...
  生成 3: ou enter the building facing east.
2. Walking west...
✓ Hidden states已保存至 method1_direction_complete_results.pkl

✅ Cell 1完成！
✅ 所有生成的hidden states已保存至 method1_direction_complete_results.pkl


In [40]:
import pickle
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity_analysis(initial_hs, path_hs_dict):
    """计算相似度分析"""
    similarity_results = {}
    
    print("="*60)
    print("相似度分析结果")
    print("="*60)
    
    for path_name, hs_list in path_hs_dict.items():
        path_similarities = []
        
        # 计算初始hs与每个路径生成hs的相似度
        for path_hs in hs_list:
            similarity = cosine_similarity([initial_hs], [path_hs])[0][0]
            path_similarities.append(similarity)
        
        # 计算统计量
        avg_similarity = np.mean(path_similarities)
        std_similarity = np.std(path_similarities)
        max_similarity = np.max(path_similarities)
        min_similarity = np.min(path_similarities)
        
        similarity_results[path_name] = {
            'average': avg_similarity,
            'std': std_similarity,
            'max': max_similarity,
            'min': min_similarity,
            'all_similarities': path_similarities
        }
        
        print(f"{path_name:20} | 平均相似度: {avg_similarity:.4f} ± {std_similarity:.4f} "
              f"(范围: {min_similarity:.4f} - {max_similarity:.4f})")
    
    return similarity_results

# 1. 加载之前保存的生成结果
with open("method1_direction_complete_results.pkl", 'rb') as f:
    generated_data = pickle.load(f)
path_hs_dict = generated_data['path_hidden_states']

# 2. 加载初始hidden states（使用方向题对应的初始数据文件）
with open("initial_hidden_states_direction.pkl", 'rb') as f:
    initial_data = pickle.load(f)
hs_A = initial_data['hs_A']

# 3. 计算相似度
print("计算相似度...")
similarity_results = calculate_similarity_analysis(hs_A, path_hs_dict)

# 4. 找出最相似的路径
best_path = max(similarity_results.items(), key=lambda x: x[1]['average'])
print(f"\n🎯 最相似路径: {best_path[0]} (平均相似度: {best_path[1]['average']:.4f})")

# 5. 相似度排名展示
print("\n" + "="*60)
print("相似度排名")
print("="*60)
sorted_results = sorted(similarity_results.items(), key=lambda x: x[1]['average'], reverse=True)
for i, (path_name, stats) in enumerate(sorted_results, 1):
    print(f"{i:2d}. {path_name:20} : {stats['average']:.4f}")

print(f"\n✅ 方法一实验全流程完成！")
print(f"✅ 初始hs_A可在后续实验中直接使用")

计算相似度...
相似度分析结果
path_1_full          | 平均相似度: 0.2637 ± 0.0506 (范围: 0.2029 - 0.3267)
path_2_full          | 平均相似度: 0.1674 ± 0.1188 (范围: 0.0522 - 0.3309)
shortcut_partial     | 平均相似度: 0.2782 ± 0.0334 (范围: 0.2501 - 0.3251)
shortcut_direct      | 平均相似度: 0.2278 ± 0.0128 (范围: 0.2114 - 0.2426)

🎯 最相似路径: shortcut_partial (平均相似度: 0.2782)

相似度排名
 1. shortcut_partial     : 0.2782
 2. path_1_full          : 0.2637
 3. shortcut_direct      : 0.2278
 4. path_2_full          : 0.1674

✅ 方法一实验全流程完成！
✅ 初始hs_A可在后续实验中直接使用


### 方法二

In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

使用设备: cuda


In [49]:
def get_logit_lens_probabilities(prompt: str, target_token: str, layer_idx: int = None):
    """
    使用Logit Lens方法获取目标token在各层的概率
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(device)  # 确保输入在正确设备上
    
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    
    # 获取所有层的hidden states
    all_hidden_states = outputs.hidden_states
    
    # 获取目标token ID
    target_token_ids = tokenizer.encode(target_token, add_special_tokens=False)
    if not target_token_ids:
        raise ValueError(f"无法编码目标token: {target_token}")
    target_token_id = target_token_ids[0]
    
    # 获取最后一token位置
    last_token_pos = inputs['attention_mask'].sum() - 1
    
    layer_probs = []
    
    for layer, hidden_states in enumerate(all_hidden_states):
        # 获取该层最后一个token的hidden state
        last_token_hidden = hidden_states[0, last_token_pos, :]  # [hidden_dim]
        
        # 使用LM Head计算logits - 确保在正确设备上
        logits = model.lm_head(last_token_hidden)  # [vocab_size]
        
        # 计算概率
        probs = torch.softmax(logits, dim=-1)
        target_prob = probs[target_token_id].item()
        
        layer_probs.append(target_prob)
    
    if layer_idx is not None:
        return layer_probs[layer_idx]
    else:
        return layer_probs

def generate_standard_curve(prompt_type: str, target_token: str, num_samples: int = 3):
    """
    生成标准曲线（多次生成取平均）
    """
    all_curves = []
    
    for i in range(num_samples):
        if prompt_type == "full":
            # 完整推理路径prompt
            prompt = f"""{XP}

Let's reason step by step:
1. Start walking west
2. First right: west → north
3. Second right: north → east  
4. Left: east → north
5. So the final direction is north.

Therefore, the answer is:"""
            
        elif prompt_type == "shortcut":
            # 纯跳步prompt
            prompt = f"""{XP}

The final direction is north.

Therefore, the answer is:"""
        else:
            raise ValueError("prompt_type必须是'full'或'shortcut'")
        
        # 获取概率曲线
        curve = get_logit_lens_probabilities(prompt, target_token)
        all_curves.append(curve)
    
    # 计算平均曲线
    all_curves = np.array(all_curves)
    avg_curve = np.mean(all_curves, axis=0)
    std_curve = np.std(all_curves, axis=0)
    
    return avg_curve, std_curve, all_curves.tolist()

def plot_probability_curves(curves_dict: Dict, title: str = "Logit Lens Probability Curves"):
    """
    绘制概率曲线对比图
    """
    plt.figure(figsize=(12, 8))
    
    colors = {'full': 'green', 'shortcut': 'red', 'test': 'blue'}
    labels = {'full': '完整推理标准曲线', 'shortcut': '纯跳步标准曲线', 'test': '待验证曲线'}
    
    for curve_name, curve_data in curves_dict.items():
        if curve_name in colors:
            if isinstance(curve_data, tuple) and len(curve_data) == 2:
                # 有误差线的数据
                avg_curve, std_curve = curve_data
                layers = range(len(avg_curve))
                plt.plot(layers, avg_curve, label=labels[curve_name], 
                        color=colors[curve_name], linewidth=2)
                plt.fill_between(layers, avg_curve - std_curve, avg_curve + std_curve,
                               alpha=0.2, color=colors[curve_name])
            else:
                # 单条曲线
                layers = range(len(curve_data))
                plt.plot(layers, curve_data, label=labels[curve_name], 
                        color=colors[curve_name], linewidth=2)
    
    plt.xlabel("Layer")
    plt.ylabel("Probability")
    plt.title(title)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

print("✅ Cell 2: 工具函数定义完成")

✅ Cell 2: 工具函数定义完成


In [50]:
# 定义目标概念（中间步骤C）
# 在这个问题中，中间概念C可以是"east"（在推理过程中出现的中间方向）
target_concept = "north"  # 我们关注最终方向"north"的出现模式

print("生成完整推理标准曲线...")
full_avg, full_std, full_samples = generate_standard_curve("full", target_concept, num_samples=3)

print("生成纯跳步标准曲线...")
shortcut_avg, shortcut_std, shortcut_samples = generate_standard_curve("shortcut", target_concept, num_samples=3)

# 保存标准曲线
standard_curves = {
    'full': (full_avg, full_std),
    'shortcut': (shortcut_avg, shortcut_std),
    'full_samples': full_samples,
    'shortcut_samples': shortcut_samples,
    'target_concept': target_concept
}

with open('standard_curves.pkl', 'wb') as f:
    pickle.dump(standard_curves, f)

# 绘制标准曲线
curves_to_plot = {
    'full': (full_avg, full_std),
    'shortcut': (shortcut_avg, shortcut_std)
}

plot_probability_curves(curves_to_plot, "标准曲线对比")

print("✅ Cell 3: 标准曲线生成完成")
print(f"完整推理曲线形状: {full_avg.shape}")
print(f"纯跳步曲线形状: {shortcut_avg.shape}")

生成完整推理标准曲线...


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [51]:
def generate_explicit_shortcut_response():
    """生成显式跳步响应"""
    
    prompt = f"""{XP}

Answer in format: Initial Condition → [direction]
Initial Condition → """
    
    inputs = tokenizer(prompt, return_tensors="pt").to(device)  # 确保在正确设备上
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=10,
            num_return_sequences=1,
            do_sample=False,
            temperature=0.0,
            pad_token_id=tokenizer.eos_token_id,
            output_hidden_states=True,
            return_dict_in_generate=True
        )
    
    response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    clean_response = response.replace(prompt, "").strip()
    
    print(f"生成的显式跳步响应: '{clean_response}'")
    
    # 保存hidden states用于后续验证 - 确保转移到CPU
    last_hidden_state = outputs.hidden_states[-1][-1][0, -1, :].cpu().numpy()
    
    return clean_response, last_hidden_state, prompt

# 生成显式跳步
print("生成显式跳步响应...")
shortcut_response, shortcut_hidden_state, shortcut_prompt = generate_explicit_shortcut_response()

# 保存结果
test_data = {
    'response': shortcut_response,
    'hidden_state': shortcut_hidden_state,
    'prompt': shortcut_prompt
}

with open('test_shortcut_data.pkl', 'wb') as f:
    pickle.dump(test_data, f)

print("✅ Cell 4: 显式跳步生成完成")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


生成显式跳步响应...


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [52]:
def generate_test_curve():
    """为待验证的显式跳步生成概率曲线"""
    
    # 使用与显式跳步相同的prompt结构
    test_prompt = f"""{XP}

Answer in format: Initial Condition → [direction]
Initial Condition → """
    
    # 获取概率曲线
    test_curve = get_logit_lens_probabilities(test_prompt, target_concept)
    
    return test_curve, test_prompt

print("生成待验证曲线...")
test_curve, test_prompt = generate_test_curve()

# 保存待验证曲线
test_results = {
    'test_curve': test_curve,
    'test_prompt': test_prompt,
    'target_concept': target_concept
}

with open('test_curve_results.pkl', 'wb') as f:
    pickle.dump(test_results, f)

print("✅ Cell 5: 待验证曲线生成完成")
print(f"待验证曲线长度: {len(test_curve)}")

生成待验证曲线...


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

### 方法三

In [18]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np
import pickle
from tuned_lens import TunedLens
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Tuple

# Set up plotting style
plt.rcParams['figure.figsize'] = [10, 6]
sns.set_style("whitegrid")

# Load model and tokenizer
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    output_hidden_states=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load Tuned Lens
print("Loading Tuned Lens...")
tuned_lens = TunedLens.from_model_and_pretrained(model, revision="main")
tuned_lens = tuned_lens.to(model.device)
print("✓ Tuned Lens loaded successfully")

# Load previous experiment data
print("Loading previous experiment data...")
with open("method1_complete_results.pkl", 'rb') as f:
    method1_data = pickle.load(f)
with open("method2_complete_results.pkl", 'rb') as f:
    method2_data = pickle.load(f)

hs_A = method1_data['initial_data']['hs_A']
concept_definitions = method2_data['concept_definitions']
summary_results = method2_data['summary_results']

print("✓ Previous data loaded successfully")
print(f"hs_A shape: {hs_A.shape}")

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct.
401 Client Error. (Request ID: Root=1-69085392-0fbc5c9017acbbe17abc2b56;3722eefe-2728-4ad9-8311-ebbfe81df568)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
def calculate_performance_score(hidden_state: np.ndarray, problem: str, expected_answer: str, 
                              layer_idx: int = -1) -> float:
    """
    Calculate performance score using Tuned Lens for multi-token answers
    """
    # Convert hidden state to tensor
    hidden_tensor = torch.tensor(hidden_state, dtype=torch.float16).to(model.device)
    hidden_tensor = hidden_tensor.unsqueeze(0)  # Add batch dimension
    
    # Tokenize the problem
    problem_inputs = tokenizer(problem, return_tensors="pt", padding=True, truncation=True)
    problem_inputs = {k: v.to(model.device) for k, v in problem_inputs.items()}
    
    # Get hidden states for the problem
    with torch.no_grad():
        problem_outputs = model(**problem_inputs, output_hidden_states=True)
    
    # Combine hs_A with problem hidden states (using the last token of the problem)
    problem_last_hidden = problem_outputs.hidden_states[layer_idx][0, -1, :]
    combined_hidden = (hidden_tensor + problem_last_hidden.unsqueeze(0)) / 2
    
    # Apply Tuned Lens transformation
    with torch.no_grad():
        transformed_hidden = tuned_lens(combined_hidden, layer_idx)
    
    # Get logits through LM head
    logits = model.lm_head(transformed_hidden)
    
    # Calculate probabilities
    probs = torch.softmax(logits, dim=-1)
    
    # Tokenize expected answer
    answer_tokens = tokenizer.encode(expected_answer, add_special_tokens=False)
    
    if not answer_tokens:
        return 0.0
    
    # Calculate performance score: average probability of correct token sequence
    performance_scores = []
    current_probs = probs[0]  # Remove batch dimension
    
    for i, correct_token in enumerate(answer_tokens):
        if i < len(current_probs):
            token_prob = current_probs[correct_token].item()
            performance_scores.append(token_prob)
    
    if not performance_scores:
        return 0.0
    
    # Return average probability across all answer tokens
    return np.mean(performance_scores)

def evaluate_hidden_state_performance(hs: np.ndarray, problem_sets: Dict[str, List[Tuple[str, str]]]) -> Dict[str, float]:
    """
    Evaluate hidden state performance on multiple problem sets
    """
    results = {}
    
    for set_name, problems in problem_sets.items():
        set_scores = []
        for problem, expected_answer in problems:
            score = calculate_performance_score(hs, problem, expected_answer)
            set_scores.append(score)
        
        results[set_name] = np.mean(set_scores)
        print(f"{set_name}: Average performance = {results[set_name]:.4f}")
    
    return results

In [ ]:
# Generate complex problems that require specific intermediate concepts
complex_problem_sets = {
    "requires_grace_remaining": [
        ("If the total work was 36 units, how many units are left after Grace finishes? Express as a number.", "24"),
        ("After Grace takes her share, what fraction of the original work remains? Express as fraction.", "2/3"),
        ("Grace does 12 units of work from the total. How many units remain? Express as a number.", "24")
    ],
    
    "requires_andrew_work": [
        ("What fraction of work would Andrew do if he took two-thirds of the remainder instead of half? Express as fraction.", "4/9"),
        ("Andrew's share is half of the work remaining after Grace. What is his fraction? Express as fraction.", "1/3"),
        ("If Andrew does 12 units from the remaining work after Grace, what was the total work? Express as a number.", "36")
    ],
    
    "requires_beatrice_work": [
        ("Beatrice does 9 units of work from the total. What is the total work? Express as a number.", "36"),
        ("What fraction of total work is done by Beatrice? Express as fraction.", "1/4"),
        ("If Beatrice's share is three times John's share, what fraction does John do? Express as fraction.", "1/12")
    ],
    
    "requires_john_final": [
        ("What fraction of the entire work does John complete? Express as fraction.", "1/12"),
        ("John's share is equal to the difference between total and all delegated work. What is it? Express as fraction.", "1/12"),
        ("After all delegations, what fraction remains for John? Express as fraction.", "1/12")
    ]
}

# Generate hidden states for intermediate concepts (A1, A2) for control group
def generate_concept_hidden_states():
    """Generate hidden states for specific intermediate concepts"""
    concept_prompts = {
        "hs_grace_remaining": f"""{XP}
Step: John delegates 1/3 to Grace, so work remaining = 2/3.
Based on this:""",
        
        "hs_andrew_work": f"""{XP}
Step: Andrew does half of the remainder after Grace: 1/2 × 2/3 = 1/3.
Based on this:""",
        
        "hs_beatrice_work": f"""{XP}
Step: Beatrice does 1/4 of all work.
Based on this:""",
        
        "hs_john_final": f"""{XP}
Step: Total delegated = 1/3 + 1/3 + 1/4 = 11/12, so John does 1 - 11/12 = 1/12.
Based on this:"""
    }
    
    concept_hidden_states = {}
    
    for concept_name, prompt in concept_prompts.items():
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
        
        with torch.no_grad():
            outputs = model(**inputs, output_hidden_states=True)
        
        hidden_state = outputs.hidden_states[-1][0, -1, :].cpu().numpy()
        concept_hidden_states[concept_name] = hidden_state
        print(f"Generated {concept_name}")
    
    return concept_hidden_states

print("Generating control hidden states...")
control_hidden_states = generate_concept_hidden_states()